In [4]:
import pygame
import random
from enum import Enum
from collections import namedtuple , deque
import numpy as np

In [5]:
pygame.init()
font = pygame.font.Font("BrownieStencil-vmrPE.ttf",25)

class Direction(Enum):
    RIGHT = 1
    LEFT = 2
    UP = 3
    DOWN = 4
    
Point = namedtuple('Point','x, y')

#rgb 😎😎🪩🕺🕺

WHITE = (255,255,255)
RED = (200,0,0)
OUTER_BODY_COLOUR = list(np.random.choice(range(256), size=3))
INNER_BODY_COLOUR = list(np.random.choice(range(256), size=3))
print(OUTER_BODY_COLOUR,INNER_BODY_COLOUR)
BLACK = (0,0,0)

BLOCK_SIZE = 20
SPPED =2000

class SankeAI():
    
    def __init__(self,game):
        self.game = game
        
    def find_path_to_food(self):
        start = self.game.head
        food = self.game.food
        visited =set()
        queue = deque([(start,[])])
        
        while queue:
            current,path = queue.popleft()
            
            if current == food:
                return path
            
            if current in visited:
                continue
            visited.add(current)
            
            for direction in [Direction.RIGHT,Direction.LEFT,Direction.UP,Direction.DOWN]:
                new_position = self._move(current,direction)
                
                if self._is_valid_move(new_position):
                    queue.append((new_position,path +[direction]))
                    
        return[]
    
    def make_decision(self):
        path_to_food = self.find_path_to_food()
        
        if path_to_food:
            return path_to_food[0]
        else:
            #if path not found RETURN random move
            
            return random.choice(list(Direction))
    
    def _move(self,point,direction):
        
        x ,y =point
        if direction == Direction.RIGHT:
            x += BLOCK_SIZE
            
        elif direction == Direction.LEFT:
            x -= BLOCK_SIZE
            
        elif direction == Direction.DOWN:
            y += BLOCK_SIZE
        
        elif direction == Direction.UP:
            y -= BLOCK_SIZE
            
        return Point(x,y) 
    
    def _is_valid_move(self,point):
        #within bounds or not
        return 0 <= point.x <self.game.w and 0<= point.y<self.game.h and point not in self.game.snake
        

    
    
    
    
    ##+++++++_________########@@@@@@@@@@@@@@@@@@+$$$$$$$$$$$$$$$@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$@
    
    
    
    
class Sankegame:
    
    def __init__(self,w=1200,h=600):
        self.w = w
        self.h = h
        
        #display 
        self.display =pygame.display.set_mode((self.w , self.h))
        pygame.display.set_caption("Sanke")
        self.clock = pygame.time.Clock()
        

        
    #___________________________GAME STATE _______________________________________________
        self.direction = Direction.RIGHT
        
        self.head = Point(self.w/2 , self.h/2)
        
        #HMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM
        self.snake = [self.head,Point(self.head.x-BLOCK_SIZE,self.head.y),Point(self.head.x-(2*BLOCK_SIZE),self.head.y)]
        
        self.score  = 0
        self.food = None 
        self._place_food()
        
    def _place_food(self):
        
        x = random.randint(0,(self.w-BLOCK_SIZE)//BLOCK_SIZE)*BLOCK_SIZE
        y =random.randint(0,(self.h-BLOCK_SIZE)//BLOCK_SIZE)*BLOCK_SIZE
        
        self.food = Point(x, y)
        if self.food in self.snake:
            self._place_food()


    
    def play_sanke(self):
        
        #collect user inputs
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                quit()
                
            if event.type ==pygame.KEYDOWN:
                
                if event.key == pygame.K_LEFT:
                    self.direction = Direction.LEFT
                    
                elif event.key == pygame.K_RIGHT:
                    self.direction = Direction.RIGHT
                    
                elif event.key == pygame.K_UP:
                    self.direction = Direction.UP
                    
                elif event.key == pygame.K_DOWN:
                    self.direction = Direction.DOWN
                    
        #move
        
        self._move(self.direction)
        self.snake.insert(0,self.head)
        #check if game over
        game_Over = False
        
        if self._is_collision():
            game_Over  = True
            return game_Over, self.score
        
        #place new food
        
        if self.head == self.food:
            self.score += 1
            self._place_food()
        else:
            self.snake.pop()
        
        #update ui and clock
        
        self._update_ui()
        self.clock.tick(SPPED)
        return game_Over , self.score
    
   
    
    def _is_collision(self):
        
        #Collision with boundary 
        if self.head.x > self.w -BLOCK_SIZE or self.head.x <0 or self.head.y > self.h-BLOCK_SIZE or self.head.y <0:
            return True
        
        #Collision with itself
        if self.head in self.snake[1:]:
            return True
        
        return False
    
    def _update_ui(self):
        self.display.fill(BLACK)
        
        for pt in self.snake:
            pygame.draw.rect(self.display,OUTER_BODY_COLOUR,pygame.Rect(pt.x,pt.y,BLOCK_SIZE,BLOCK_SIZE))
            pygame.draw.rect(self.display,INNER_BODY_COLOUR,pygame.Rect(pt.x+4,pt.y+4,12,12))
            
        pygame.draw.rect(self.display,RED,pygame.Rect(self.food.x,self.food.y,BLOCK_SIZE,BLOCK_SIZE))
        
        text = font.render("Score:"+str(self.score),True,WHITE)
        self.display.blit(text,[0,0])
        pygame.display.flip()
        
    def _move(self,direction):
        x= self.head.x
        y = self.head.y
        if direction == Direction.RIGHT:
            x += BLOCK_SIZE
            
        elif direction == Direction.LEFT:
            x -= BLOCK_SIZE
        
        elif direction == Direction.DOWN:
            y += BLOCK_SIZE
            
        elif direction == Direction.UP:
            y -= BLOCK_SIZE
            
        self.head = Point(x, y)
        
if __name__ == '__main__':
    game = Sankegame()
    snake_ai = SankeAI(game)

    while True:
        game_over, score = game.play_sanke()

        if game_over:
            break

        # Uncomment the following line to enable AI control
        game.direction = snake_ai.make_decision()

        print('Final Score: {}'.format(score))
            
        
        

[34, 163, 229] [67, 67, 182]
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 0
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 1
Final Score: 2
Final Score: 2
Final Score: 2
Final Score: 2
Final Score: 2
Final Score: 2
Final Score